In [1]:
!python3 -V

Python 3.8.15


In [2]:
!pip3 install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install fastai==2.2.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:56tcmalloc: large alloc 1147494400 bytes == 0x393f6000 @  0x7f27d6698615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████████████████████████████▏  | 1055.7 MB 1.2 MB/s eta 0:01:26tcmalloc: large alloc 1434370048 bytes == 0x7da4c000 @  0x7f27d6698615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |█████

In [3]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from fastai.text.all import *
import gc

In [4]:
print(torch. __version__)

1.7.1+cu110


In [5]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [6]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    random.seed(seed_value) # Python
    if use_cuda: 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
#Remember to use num_workers=0 when creating the DataBunch.

In [7]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [8]:
!ls /content/drive/MyDrive/YelpDataset/dataframes

AutoDF.pkl		     HealthMedicalDF.pkl  restaurantDF.pkl
beautyandSpaDF.pkl	     HomeServicesDF.pkl   shoppingDF.pkl
EventPlanningServicesDF.pkl  LocalServicesDF.pkl
foodDF.pkl		     NightlifeDF.pkl


In [9]:
%cp /content/drive/MyDrive/YelpDataset/dataframes/AutoDF.pkl /content/

In [10]:
import pandas as pd
df = pd.read_pickle('AutoDF.pkl')

In [11]:
df = df.rename(columns={'text': 'reviewText'})
df = df.rename(columns={'stars': 'overall'})
df = df.rename(columns={'categories': 'category'})

In [12]:
df = df[['business_id','name','city','state','overall','category','reviewText']]

In [13]:
df.head(1)

,business_id,name,city,state,overall,category,reviewText
106213,rBmpy_Y1UbBx8ggHlyb7hA,Arizona Truck Outfitters,Tucson,AZ,4.5,Automotive,"I went in to get shocks for my shell and not only did they have them in stock, the lovely Kim installed them for me! She was extremely professional and very friendly. I would recommend them to anyone."


In [14]:
df['sentenceLength'] = df['reviewText'].str.count(' ') + 1

In [15]:
df.shape

(184986, 8)

In [16]:
df.head(2)

,business_id,name,city,state,overall,category,reviewText,sentenceLength
106213,rBmpy_Y1UbBx8ggHlyb7hA,Arizona Truck Outfitters,Tucson,AZ,4.5,Automotive,"I went in to get shocks for my shell and not only did they have them in stock, the lovely Kim installed them for me! She was extremely professional and very friendly. I would recommend them to anyone.",40
106224,nUqrF-h9S7myCcvNDecOvw,Iron Horse Auto Body,Santa Barbara,CA,4.5,Automotive,The absolute best service and results I've had & I have several vehicles. HIGHLY recommend!,16


In [17]:
sample_df = df.sample(frac=0.05, random_state=2021).dropna().reset_index()

In [18]:
sample_df.shape

(9249, 9)

In [19]:
del df
gc.collect()

35

In [20]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [21]:
learn = Learner(dls=None, model=model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

### Generated examples of Toys_and_Games_5

In [22]:
!ls /content/drive/MyDrive/YelpDataset/models

gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-BeautySpa.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Food.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-HomeServices.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Restaurants.pth
gpt2-finetuned-yelp-reviews-sample-1cycle-Shopping.pth


In [23]:
%cp /content/drive/MyDrive/YelpDataset/models/gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive.pth /content/

In [25]:
learn.load("gpt2-finetuned-yelp-reviews-sample-1cycle-Automotive").cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [26]:
"""N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 10
BINS = list(range(10,15))"""
N_INITIAL_WORDS = 5
LEN_SAMPLE_DF = sample_df.shape[0]
N_REVIEWS = 2000
BINS = list(range(10,351))

In [27]:
import warnings
warnings.filterwarnings(action="ignore")
from sklearn.model_selection import train_test_split
generated_df_rows = []
for i in range(len(BINS)-1):
    sample_row_df = sample_df[(sample_df["sentenceLength"]>=BINS[i]) & (sample_df["sentenceLength"]<BINS[i+1])]
    bin_proportion = (sample_row_df.shape[0])/float(LEN_SAMPLE_DF)
    n_reviews_to_generate = int((bin_proportion)*N_REVIEWS)+1
    try:
        or_df, cg_df =  train_test_split(sample_row_df, test_size=0.5, random_state=2021)
    except Exception as e:
        print(e)
        continue
    sample_reviews_df = cg_df.sample(n=n_reviews_to_generate,random_state=2021)
    or_sample_reviews_df = or_df.sample(n=n_reviews_to_generate,random_state=2021)
    if BINS[i] in range(10,21):
        max_len = 20
    else:
        max_len = BINS[i]
    for index, row in sample_reviews_df.iterrows():
        text = row["reviewText"]
        prompt = (" ").join(text.strip().split(" ")[:N_INITIAL_WORDS])
        prompt_ids = tokenizer.encode(prompt)
        inp = tensor(prompt_ids)[None].cuda()
        # set seed to reproduce results
        random_seed(seed_value=2021, use_cuda=True)
        preds = learn.model.generate(inp, max_length=max_len, do_sample=True, top_k=0, top_p=0.92, temperature=0.7)
        decoded_preds = tokenizer.decode(preds[0],skip_special_tokens=True)
        generated_df_rows.append([row["category"],row["overall"],decoded_preds,"CG"])
    for index, row in or_sample_reviews_df.iterrows():
        generated_df_rows.append([row["category"],row["overall"],row["reviewText"],"OR"])
#     break
generated_df = pd.DataFrame(generated_df_rows, columns=["category", "rating", "text", "label"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

...

With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [28]:
generated_df.shape

(4114, 4)

In [29]:
generated_df.head()

,category,rating,text,label
0,Automotive,3.0,They treat their customer like family. The guys here are knowledgeable and friendly. They have been taking care,CG
1,Automotive,4.0,Great work and service drove me to the dealership to pick up my car. I was treated fairly,CG
2,Automotive,3.5,Quick and friendly service for tire rotation and multipoint inspection.,OR
3,Automotive,2.5,The situation was promptly corrected. Thank you Courtesy Collision.,OR
4,Automotive,2.5,"Rude beond belief! \nQuestionable skills, an unprofessional attitude. I will never",CG


In [30]:
def post_process(str, k=3):
    if not str.endswith("."):
        full_stop_split_str = str.split(".")
        clean_full_stop_split_str = [s.strip() for s in full_stop_split_str if s.strip()]
        if len(clean_full_stop_split_str) > 1:
            last_sentence = clean_full_stop_split_str[-1]
        else:
            return str
        last_sentence_len = len(last_sentence.split(" "))
        if last_sentence_len > k:
            return str
        return (". ").join(clean_full_stop_split_str[:-1]).strip()+(".")
    return str

In [31]:
generated_df['text_'] = generated_df['text'].apply(lambda x: post_process(x))

In [32]:
generated_df.head()

,category,rating,text,label,text_
0,Automotive,3.0,They treat their customer like family. The guys here are knowledgeable and friendly. They have been taking care,CG,They treat their customer like family. The guys here are knowledgeable and friendly. They have been taking care
1,Automotive,4.0,Great work and service drove me to the dealership to pick up my car. I was treated fairly,CG,Great work and service drove me to the dealership to pick up my car. I was treated fairly
2,Automotive,3.5,Quick and friendly service for tire rotation and multipoint inspection.,OR,Quick and friendly service for tire rotation and multipoint inspection.
3,Automotive,2.5,The situation was promptly corrected. Thank you Courtesy Collision.,OR,The situation was promptly corrected. Thank you Courtesy Collision.
4,Automotive,2.5,"Rude beond belief! \nQuestionable skills, an unprofessional attitude. I will never",CG,"Rude beond belief! \nQuestionable skills, an unprofessional attitude."


In [33]:
generated_df.drop(columns=["text"],axis=1,inplace=True)

In [40]:
generated_df[100:112]

,category,rating,label,text_
100,Automotive,3.5,OR,"the best oil change experience. friendly employees, quick turn around time, fair prices (there's always coupons). highly recommend."
101,Automotive,4.5,OR,This car repair shop is well done and the workplace is clean. Mr. Lang and Anh Thanh.
102,Automotive,2.5,OR,Sales and service departments have exceeded my expectations. The best car buying and service experience I've ever had.
103,Automotive,2.0,OR,Love coming here I think is one of the best car wash in town the staff always friendly!!
104,Automotive,5.0,CG,Very pleased with the results. \n\nI'd recommend this location to anyone who needs their vehicle
105,Automotive,5.0,CG,"Expert, honest auto technicians. I've had my car serviced here twice now and each time"
106,Automotive,1.5,CG,Unkompliziert und sehr schnell\nBin sehr wen I
107,Automotive,3.5,CG,Great place to buy a car. The guys here are knowledgeable and friendly. They have been around for
108,Automotive,3.5,CG,"Awesome folks. Let me tell you, they are honest, reliable and fair."
109,Automotive,3.0,CG,Defiantly a better one then what I was hoping for. I will definitely be back!Very


In [35]:
generated_df.to_csv("Generated_Sentences_Auto.csv", index=None)

In [36]:
%cp /content/Generated_Sentences_Auto.csv /content/drive/MyDrive/YelpDataset/generatedData